In [2]:
# Import boto3
import boto3

# Create a low-level EC2 client
ec2_client = boto3.client('ec2', region_name='us-east-1')

# Create a low-level ELB client
elb_client = boto3.client('elbv2')

# Create a low-level Auto Scaling client
as_client = boto3.client('autoscaling')

In [4]:
# Step 1: Create VPC

vpc_response = ec2_client.create_vpc(CidrBlock='10.0.0.0/16')
vpc_id = vpc_response['Vpc']['VpcId']

# Tag the VPC
ec2_client.create_tags(Resources=[vpc_id], Tags=[{'Key': 'Name', 'Value': 'demo_vpc'}])

# Enable DNS support and hostnames (useful for EC2 instances)
ec2_client.modify_vpc_attribute(VpcId=vpc_id, EnableDnsSupport={'Value': True})
ec2_client.modify_vpc_attribute(VpcId=vpc_id, EnableDnsHostnames={'Value': True})

# Create an internet gateway and attach to the VPC
igw_response = ec2_client.create_internet_gateway()
igw_id = igw_response['InternetGateway']['InternetGatewayId']
ec2_client.attach_internet_gateway(InternetGatewayId=igw_id, VpcId=vpc_id)

{'ResponseMetadata': {'RequestId': 'f3ebfbd7-2b10-4bfb-b458-8bc3573ad0ed',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f3ebfbd7-2b10-4bfb-b458-8bc3573ad0ed',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '231',
   'date': 'Wed, 16 Apr 2025 07:27:24 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [5]:
# Create Subnets

# Subnet 01
subnet_response = ec2_client.create_subnet(
    VpcId=vpc_id,
    CidrBlock='10.0.1.0/24',
    AvailabilityZone='us-east-1a'
)

subnet_id_01 = subnet_response['Subnet']['SubnetId']

# Subnet 02
subnet_response = ec2_client.create_subnet(
    VpcId=vpc_id,
    CidrBlock='10.0.2.0/24',
    AvailabilityZone='us-east-1b'
)

subnet_id_02 = subnet_response['Subnet']['SubnetId']

In [6]:
# Route Table

rt_response = ec2_client.create_route_table(VpcId=vpc_id)

rtd_id = rt_response['RouteTable']['RouteTableId']

ec2_client.create_route(
    RouteTableId=rtd_id,
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igw_id

)

ec2_client.associate_route_table(RouteTableId=rtd_id, SubnetId=subnet_id_01)

ec2_client.associate_route_table(RouteTableId=rtd_id, SubnetId=subnet_id_02)

{'AssociationId': 'rtbassoc-023b3f69317e0093d',
 'AssociationState': {'State': 'associated'},
 'ResponseMetadata': {'RequestId': 'a0e17c08-b100-4c73-9009-e9ea188ff159',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a0e17c08-b100-4c73-9009-e9ea188ff159',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '325',
   'date': 'Wed, 16 Apr 2025 07:31:26 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [7]:
# Modify the subnets to auto-assign public IPv4 address(optional)

ec2_client.modify_subnet_attribute(
    SubnetId=subnet_id_01,
    MapPublicIpOnLaunch={'Value': True}

)


ec2_client.modify_subnet_attribute(
    SubnetId=subnet_id_02,
    MapPublicIpOnLaunch={'Value': True}

)

{'ResponseMetadata': {'RequestId': '25db6898-c4c1-456c-9082-664a0f874bb6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '25db6898-c4c1-456c-9082-664a0f874bb6',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '231',
   'date': 'Wed, 16 Apr 2025 07:31:54 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [11]:
# Fetching the subnet IDs

subnet_response = ec2_client.describe_subnets(
    Filters = [
        {
        'Name': 'vpc-id',
        'Values': [vpc_id]
        }
        

    ]


)

for subnet in subnet_response['Subnets']:
    print(subnet['SubnetId'])

subnet-0a40b589953926892
subnet-01cb54ae4a28ab7e6


In [16]:
# Create a security group and add inbound rules to it

group_name = 'mydemodcigroup'
description = 'Allows SSH and HTTP access from anywhere'

try:
    # Step 1: Create the security group
    response_sg = ec2_client.create_security_group(GroupName=group_name, 
                                     Description=description, 
                                     VpcId=vpc_id)
    security_group_id = response_sg['GroupId']
    print(f"Security Group Created: {security_group_id }")

    # Step 2: Define the inbound rules (Allow SSH and HTTP)
    ingress_rules = [

    {
        'IpProtocol': 'tcp',
        'FromPort': 22,  # SSH
        'ToPort': 22,
        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]

    },

    {
        'IpProtocol': 'tcp',
        'FromPort': 80,  # HTTP
        'ToPort': 80,
        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]

    }


]
    # Step 3: Adding the inbound rules to the security group
    ec2_client.authorize_security_group_ingress(GroupId=security_group_id, 
                                                IpPermissions=ingress_rules)
    
except:
    print("There seems to be some issue while creating a security group and/or adding rules to it.")
else:
    print("We have created a security group and added the inbound rules to it.")


Security Group Created: sg-0ba22b13d4b8906fb
We have created a security group and added the inbound rules to it.


In [17]:
# Create a new key pair

key_pair = ec2_client.create_key_pair(KeyName='capboto3-kp')

In [18]:
# Create a launch template

import base64

user_data_script = '''#!/bin/bash
yum update -y
yum install httpd -y
service httpd start
chkconfig httpd on
cd /var/www/html
echo "<html><h1>Congratulations! You've just deployed your first web server on an Amazon EC2 instance.</h1></html>" > index.html'''


# UserData must be Base64-encoded
encoded = base64.b64encode(user_data_script.encode('utf-8')).decode('utf-8')

response_lt = ec2_client.create_launch_template(
    LaunchTemplateName='capstone-lt',
    LaunchTemplateData={
        'ImageId': 'ami-00a929b66ed6e0de6',  # Amazon Linux 2023 AMI
        'InstanceType': 't2.micro',
        'KeyName': 'capboto3-kp',
        'SecurityGroupIds': ['sg-0ba22b13d4b8906fb'],
        'UserData': encoded

    }

)

ltd_id = response_lt['LaunchTemplate']['LaunchTemplateId']

In [28]:
print(ltd_id)

lt-0e947ff894305f52e


>
> _________________________________________
> ### Registration of targets
>
> -  No need to register the targets manually as the instances will be launched by Auto Scaling
>
> - Upon launching, the instances will get registered to the LB automatically.
>
> - This is because the LB target group and AS group overlap.
>
> _________________________________________
>
>

In [24]:
# Creating a load balancer target group
response_tg = elb_client.create_target_group(
    Name='capstonetargetgroup',
    TargetType='instance',
    Protocol='HTTP',
    Port=80,
    VpcId=vpc_id,
    HealthCheckProtocol='HTTP',
    HealthCheckPort='80',
    HealthCheckPath='/index.html'

)

# Extracting the TG ARN and setting a variable for it
for target in response_tg['TargetGroups']:
    target_arn = target['TargetGroupArn']


# Create a load balancer
elb_response = elb_client.create_load_balancer(
    Name='capstonealb',
    Scheme='internet-facing',
    IpAddressType='ipv4',
    Subnets=['subnet-0a40b589953926892', 'subnet-01cb54ae4a28ab7e6'],
    SecurityGroups=['sg-0ba22b13d4b8906fb'],
    Type='application'
    
)

# Extracting the LB's ARN and setting a variable for it
for lb in elb_response['LoadBalancers']:
    lb_arn = lb['LoadBalancerArn']


# Create a listener
elb_client.create_listener(
    LoadBalancerArn=lb_arn,
    Protocol='HTTP',
    Port=80,
    DefaultActions=[
        {
            'Type': 'forward',
            'TargetGroupArn': target_arn



        }
    ]


)

print("Using boto3, we've launched an ALB successfully.")


Using boto3, we've launched an ALB successfully.


In [25]:
# Creating an Auto Scaling Group

response_asg = as_client.create_auto_scaling_group(
    AutoScalingGroupName='capstoneasg',
    LaunchTemplate={
        'LaunchTemplateId': ltd_id ,
        'Version': '$Latest'
    },
    VPCZoneIdentifier='subnet-0a40b589953926892, subnet-01cb54ae4a28ab7e6', # Comma-separated subnet IDs,
    TargetGroupARNs=[target_arn], # Specify the LB's target group
    HealthCheckType='EC2',
    HealthCheckGracePeriod=300,
    MinSize=2,
    MaxSize=3,
    DesiredCapacity=2
)


# Create a target tracking policy
response = as_client.put_scaling_policy(
    AutoScalingGroupName='capstoneasg',
    PolicyName='cpu-tracking-policy',   # custom name assigned to the policy
    PolicyType='TargetTrackingScaling',
    TargetTrackingConfiguration={
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'ASGAverageCPUUtilization'
        },
        'TargetValue': 50.0, # Target CPU %
        'DisableScaleIn': False


    }

)




In [ ]:
# Clean Up

# Delete the Auto Scaling Group (Force Delete)
# Force Delete = True, if the instances are running
as_client.delete_auto_scaling_group(AutoScalingGroupName='capstoneasg',
                                              ForceDelete=True)

# Delete the load balancer
elb_client.delete_load_balancer(LoadBalancerArn=lb_arn)

# Deleting the launch template
ec2_client.delete_launch_template(LaunchTemplateId=ltd_id)

# Delete the security group
ec2_client.delete_security_group(GroupId='sg-0ba22b13d4b8906fb')

# Delete the subnets
ec2_client.delete_subnet(SubnetId=subnet_id_01)
ec2_client.delete_subnet(SubnetId=subnet_id_02)

# Delete the route 
ec2_client.delete_route(
    RouteTableId=rtd_id,
    DestinationCidrBlock='0.0.0.0/0'
)

# Delete the route table
ec2_client.delete_route_table(RouteTableId=rtd_id)

# Detach the internet gateway from the VPC
ec2_client.detach_internet_gateway(InternetGatewayId=igw_id, VpcId=vpc_id)

# Delete the internet gateway
ec2_client.delete_internet_gateway(InternetGatewayId=igw_id)

# Finally, delete the VPC
ec2_client.delete_vpc(VpcId=vpc_id)

